Doc chain to generate answer from llm based on provided context and query asked
Retrieval chain for getting answer by comparing closeness of vector from the stored data (KNN concept)

Ret+Doc chain concept see below for all 3 concepts of doc,ret,doc+ret

Complete Basic RAG flow

In [ ]:
import os

import bs4
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB

In [ ]:
#data ingestion
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [ ]:
docs=loader.load()
docs

In [ ]:
#data transformation
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splitted=splitter.split_documents(docs)
splitted

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings

In [ ]:
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(splitted,embeddings)
db

In [ ]:
#asking for an answer to the query
query="LangSmith has two usage limits: total traces and extended"
result=db.similarity_search(query)
result[1].page_content

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
llm=GoogleGenerativeAI(model="gemini-1.5-flash")
llm

now we saw that in prompt we can design our llm to act as a specific " engg or a thing" while answering the queries
now to get output from a specific content we can use document chain

In [ ]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. and it can help us to track our billings also")]
})

now we gave our own context and asked a query based on it the llm answered this is our document_chain
and
earlier we used retreiver to get answer of the query from the stored data

 Q : What if we want a llm to generate an answer for the query from the stored data??

     this needs to used retriever to get relevant data using vector close distance
     then document chain to get all that data and generate an answer that relates to query

we asked query--->goes to retriever ->>> search in vectore store first--> get all the relevant data ->feed the doc chain with that output data  and question  to get the output

In [ ]:
db

In [ ]:
retriever=db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

In [ ]:
#getting the response from the llm
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

In [ ]:
response['context']

In [ ]:
#provides context +answer both
response